In [1]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from PIL import Image
from transformers import TrainerCallback, LlavaNextImageProcessor
from unsloth import FastVisionModel 
from trl import SFTTrainer, SFTConfig
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
import pandas as pd
from sklearn.model_selection import train_test_split


class FinetuneQwenVL:
    def __init__(self, 
                 data,
                 epochs=1, 
                 learning_rate=1e-4,
                 warmup_ratio=0.1,
                 gradient_accumulation_steps=64,
                 optim="adamw_torch",
                 model_id="unsloth/Qwen2-VL-7B-Instruct", 
                 peft_r=8,
                 peft_alpha=16,
                 peft_dropout=0.05,
                ):
        self.epochs = epochs
        self.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
        self.model_id = model_id

        self.base_model, self.tokenizer = FastVisionModel.from_pretrained(
            model_name = self.model_id,
            load_in_4bit = False,
            use_gradient_checkpointing = "unsloth",
        )
        self.model = FastVisionModel.get_peft_model(
            self.base_model,
            finetune_vision_layers     = True, # False if not finetuning vision layers
            finetune_language_layers   = True, # False if not finetuning language layers
            finetune_attention_modules = True, # False if not finetuning attention layers
            finetune_mlp_modules       = True, # False if not finetuning MLP layers
            r = peft_r,           
            lora_alpha = peft_alpha,  
            lora_dropout = peft_dropout,
            bias = "none",
            random_state = 3407,
            use_rslora = False,  
            loftq_config = None
        )
        self.learning_rate = learning_rate
        self.warmup_ratio = warmup_ratio
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.optim = optim
        self.data = data
    
    def format_data(self, row):
        image_processor = LlavaNextImageProcessor.from_pretrained(self.model_id) # Use the correct model name
        
        
        image_path = "../" + row["image"]
        input_text = row['input']
        output_text = row['output']
        try:
            image = Image.open(image_path).convert("RGB")
            image = image.resize((336, 336))
            processed_images = image_processor(images=image, return_tensors="pt") 
        except Exception as e:
            raise FileNotFoundError(f"Unable to load image at path: {image_path}. Error: {e}")

        return {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": input_text,
                        },
                        {
                            "type": "image",
                            "image": processed_images.pixel_values[0],  
                        }
                    ],
                },
                {
                    "role": "assistant",
                    "content": [
                        {
                            "type": "text",
                            "text": output_text,
                        }
                    ],
                },
            ],
        }

    def run(self):
        """
        Executes the fine-tuning process.
        """
        converted_dataset = [self.format_data(row) for row in self.data]
        converted_dataset = converted_dataset
        training_args = SFTConfig(
            learning_rate=self.learning_rate,
            output_dir='../model_cp_qutip_gs',
            optim=self.optim,
            logging_steps=1,
            report_to="none",
            fp16 = not is_bf16_supported(),
            bf16 = is_bf16_supported(),
            logging_first_step=True,
            warmup_ratio=self.warmup_ratio,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            logging_dir='./logs',
            gradient_accumulation_steps=self.gradient_accumulation_steps,
            num_train_epochs=self.epochs,
            weight_decay = 0.01,            # Regularization term for preventing overfitting
            lr_scheduler_type = "linear",   # Chooses a linear learning rate decay
            seed = 3407,
            logging_strategy = "steps",
            # load_best_model_at_end = True,
            # You MUST put the below items for vision finetuning:
            remove_unused_columns = False,
            dataset_text_field = "",
            dataset_kwargs = {"skip_prepare_dataset": True},
            dataset_num_proc = 4,
            max_seq_length = 2048,
        )
        FastVisionModel.for_training(self.model)
        
        trainer = SFTTrainer(
            model = self.model,
            tokenizer = self.tokenizer,
            data_collator = UnslothVisionDataCollator(self.model, self.tokenizer), # Must use!
            train_dataset = converted_dataset,
            args = training_args,
        )
        trainer.train(resume_from_checkpoint=False)


/home/cqilab/anaconda3/envs/llmfinetune/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
CSV_FILENAME = '../metadata-qutip-grayscale-updated.csv' 
data = pd.read_csv(CSV_FILENAME)

required_columns = ['type', 'image', 'ground_truth', 'prompt']
for column in required_columns:
    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in the CSV file.")

# Shuffle the dataset with a controlled random state
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Split data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

train_data = train_data.dropna().reset_index(drop=True)


x_train = train_data['type'][:]
images = train_data['image'][:]
y_train = train_data['ground_truth'][:]
prompts = train_data['prompt'][:]

fine_tune_data = []
for i in range(len(x_train)):
    # print the index for debugging
    fine_tune_data.append({
        "image": images[i],
        "input": prompts[i] + x_train[i],
        "output": y_train[i],
    })

finetuner = FinetuneQwenVL(
    data=fine_tune_data,
    epochs=10,
    learning_rate=5e-6,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    optim="adamw_torch_fused",
    # model_id="unsloth/Qwen2-VL-7B-Instruct",
    model_id="unsloth/llava-v1.6-mistral-7b-hf",
    peft_r=32,
    peft_alpha=32,
    peft_dropout=0.0,
)

finetuner.run()

==((====))==  Unsloth 2024.12.11: Fast Llava_Next vision patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.684 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.04s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


KeyboardInterrupt: 